In [262]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [263]:
#importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np

In [264]:
#importing ml libraries
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score,accuracy_score

In [265]:
path='/content/drive/MyDrive/data sets/train-data.csv'
dfz=pd.read_csv(path)

In [266]:
# Removing column New_price and cleaning the data
# Expanding Name into company name and car name and removing the preexisting name column
# Removing the string part of mileage, Engine and power and making the data type into float

In [267]:
def wrangle(filepath):
  df=pd.read_csv(filepath)
  df.drop(columns='New_Price',inplace=True)
  df=df.dropna()

  #Creating New Column
  df['Company_Name']=df['Name'].str.split(' ').str[0]
  df['Car_Name']=df['Name'].str.split(' ').str[1]

  #Mileage
  df['Mileage']=df['Mileage'].str.split(' ').str[0]
  df['Mileage']=df['Mileage'].astype(float)

  #Engine
  df['Engine']=df['Engine'].str.split(" ").str[0]
  df['Engine']=df['Engine'].astype(int)

  #Power
  df['Power']=df['Power'].str.split(' ').str[0]

  #Dropping Columns
  df.drop(columns='Unnamed: 0',inplace=True)
  df.drop(columns='Name',inplace=True)

  #Quantile
  low,high=df['Price'].quantile([0.05,0.95])
  mask_price=df['Price'].between(low,high)
  df=df[mask_price]

  low,high=df['Kilometers_Driven'].quantile([0.05,0.95])
  mask_distance=df['Kilometers_Driven'].between(low,high)
  df=df[mask_distance]

  #Rearranging the Data of Price to remove outlyers from the data
  df['Price']=np.log(df['Price'])

  return df

In [268]:
df=wrangle('/content/drive/MyDrive/data sets/train-data.csv')

In [269]:
df.shape

(4843, 13)

In [270]:
df.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company_Name,Car_Name
0,Mumbai,2010,72000,CNG,Manual,First,26.60,998,58.16,5.0,0.559616,Maruti,Wagon
1,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,2.525729,Hyundai,Creta
2,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.7,5.0,1.504077,Honda,Jazz
3,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,1.791759,Maruti,Ertiga
4,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.8,5.0,2.875822,Audi,A4


In [271]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4843 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           4843 non-null   object 
 1   Year               4843 non-null   int64  
 2   Kilometers_Driven  4843 non-null   int64  
 3   Fuel_Type          4843 non-null   object 
 4   Transmission       4843 non-null   object 
 5   Owner_Type         4843 non-null   object 
 6   Mileage            4843 non-null   float64
 7   Engine             4843 non-null   int64  
 8   Power              4843 non-null   object 
 9   Seats              4843 non-null   float64
 10  Price              4843 non-null   float64
 11  Company_Name       4843 non-null   object 
 12  Car_Name           4843 non-null   object 
dtypes: float64(3), int64(3), object(7)
memory usage: 529.7+ KB


In [272]:
x_train=df.drop(columns='Price')

In [273]:
y_train=df['Price']

In [274]:
x_train

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Company_Name,Car_Name
0,Mumbai,2010,72000,CNG,Manual,First,26.60,998,58.16,5.0,Maruti,Wagon
1,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,Hyundai,Creta
2,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.7,5.0,Honda,Jazz
3,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,Maruti,Ertiga
4,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.8,5.0,Audi,A4
...,...,...,...,...,...,...,...,...,...,...,...,...
6014,Delhi,2014,27365,Diesel,Manual,First,28.40,1248,74,5.0,Maruti,Swift
6015,Jaipur,2015,100000,Diesel,Manual,First,24.40,1120,71,5.0,Hyundai,Xcent
6016,Jaipur,2012,55000,Diesel,Manual,Second,14.00,2498,112,8.0,Mahindra,Xylo
6017,Kolkata,2013,46000,Petrol,Manual,First,18.90,998,67.1,5.0,Maruti,Wagon


In [275]:
y_train

0       0.559616
1       2.525729
2       1.504077
3       1.791759
4       2.875822
          ...   
6014    1.558145
6015    1.386294
6016    1.064711
6017    0.974560
6018    0.916291
Name: Price, Length: 4843, dtype: float64

In [276]:
from sklearn. preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge

In [277]:
ohe=OneHotEncoder()

In [278]:
ohe.fit(x_train)

XT_train = ohe.transform(x_train)
print(XT_train.shape)

(4843, 3633)


In [279]:
from sklearn.pipeline import make_pipeline

In [280]:
model=make_pipeline(
    OneHotEncoder(),
    LinearRegression()
)
model.fit(x_train,y_train)

Pipeline(steps=[('onehotencoder', OneHotEncoder()),
                ('linearregression', LinearRegression())])

In [281]:
y_prediction=model.predict(x_train)

In [282]:
print('R2_score',r2_score(y_train,y_prediction))

R2_score 0.9871799543890728


In [283]:
print('Mean_Absoute_Error',mean_absolute_error(y_train,y_prediction))

Mean_Absoute_Error 0.04005633329017309
